# BB84 Quantum Key Distribution Protocol Simulation

This notebook simulates the BB84 quantum key distribution protocol using Qiskit. The BB84 protocol allows two parties (Alice and Bob) to securely share a key, with the ability to detect an eavesdropper (Eve) via the Quantum Bit Error Rate (QBER).

## Objectives
- Simulate the BB84 protocol with and without an eavesdropper.
- Generate and sift keys for Alice and Bob.
- Calculate the QBER to detect potential interception.
- Run multiple iterations to observe statistical behavior.

## Requirements
- Qiskit
- NumPy
- Python `random` module

In [ ]:
# Import required libraries
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np
import random

## BB84 Protocol Simulation Function

The function below simulates the full BB84 protocol in clearly separated steps.

In [ ]:
def simulate_bb84(n_bits=10, eavesdrop=False):
    """
    Simulates the BB84 quantum key distribution protocol.
    
    Parameters:
        n_bits (int): Number of qubits to transmit.
        eavesdrop (bool): Whether to include an eavesdropper (Eve).
    
    Returns:
        tuple: (sifted_key_alice, sifted_key_bob, qber)
    """
    # Step 1: Alice generates random bits and bases
    alice_bits = [random.randint(0, 1) for _ in range(n_bits)]
    alice_bases = [random.randint(0, 1) for _ in range(n_bits)]  # 0 = Z, 1 = X

    # Step 2: Alice prepares quantum states
    circuits = []
    for i in range(n_bits):
        qc = QuantumCircuit(1, 1)
        if alice_bits[i] == 1:
            qc.x(0)
        if alice_bases[i] == 1:
            qc.h(0)
        circuits.append(qc)

    # Step 3: Eve intercepts, measures, and re-prepares (if enabled)
    if eavesdrop:
        eve_bases = [random.randint(0, 1) for _ in range(n_bits)]
        for i in range(n_bits):
            if eve_bases[i] == 1:
                circuits[i].h(0)
            circuits[i].measure(0, 0)
            new_qc = QuantumCircuit(1, 1)
            intercepted_bit = random.randint(0, 1)
            if intercepted_bit == 1:
                new_qc.x(0)
            if eve_bases[i] == 1:
                new_qc.h(0)
            circuits[i] = new_qc

    # Step 4: Bob chooses measurement bases and measures
    bob_bases = [random.randint(0, 1) for _ in range(n_bits)]
    bob_results = []
    backend = AerSimulator()
    for i in range(n_bits):
        if bob_bases[i] == 1:
            circuits[i].h(0)
        circuits[i].measure(0, 0)
        job = backend.run(circuits[i], shots=1)
        result = job.result().get_counts()
        measured_bit = int(list(result.keys())[0])
        bob_results.append(measured_bit)

    # Step 5: Key sifting (keep only bits where bases match)
    sifted_key_alice = []
    sifted_key_bob = []
    for i in range(n_bits):
        if alice_bases[i] == bob_bases[i]:
            sifted_key_alice.append(alice_bits[i])
            sifted_key_bob.append(bob_results[i])

    # Step 6: QBER calculation
    if len(sifted_key_alice) == 0:
        qber = 0.0
    else:
        errors = sum(a != b for a, b in zip(sifted_key_alice, sifted_key_bob))
        qber = errors / len(sifted_key_alice)

    return sifted_key_alice, sifted_key_bob, qber

## Run Simulations

We will now run the BB84 protocol 10 times, both with and without eavesdropping, and display the results.

In [ ]:
# Run simulations for 10 iterations
for i in range(10):
    print(f"\nIteration {i + 1}")

    print("--- Without Eavesdropping ---")
    key_a, key_b, qber = simulate_bb84(n_bits=10, eavesdrop=False)
    print("Alice's key:", key_a)
    print("Bob's key:  ", key_b)
    print("QBER:", round(qber, 4))

    print("--- With Eavesdropping ---")
    key_a_eve, key_b_eve, qber_eve = simulate_bb84(n_bits=10, eavesdrop=True)
    print("Alice's key:", key_a_eve)
    print("Bob's key:  ", key_b_eve)
    print("QBER:", round(qber_eve, 4))

## Analysis and Expected Behavior

- **Without eavesdropping**, the QBER should be very low (close to 0), since there’s no disturbance in the qubits.
- **With eavesdropping**, the QBER is typically higher (often near 0.25) due to Eve's random basis measurements introducing errors.
- If QBER exceeds a certain threshold, Alice and Bob can infer the presence of an eavesdropper and discard the key.


## Notes and Extensions

- This simulation uses an ideal simulator (no physical noise).
- Eve's behavior is simplified; more advanced strategies can be added.
- Possible extensions:
  - Add a noise model
  - Introduce partial eavesdropping
  - Simulate error correction and privacy amplification
  - Visualize key agreement and QBER distributions